<a href="https://colab.research.google.com/github/totvslabs/carol-notebooks/blob/main/notebooks/PublishCarolApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Publish Carol App
`This notebook will publish a Carol App in the given tenants after successful installation. This is a complement to notebook "Subscribe Carol App" and will use the same json file format.`
`The script will ask for the following json on execution:`

```python
{
    "authentication_config" : {
        "username": "username@totvs.com.br",
        "password": "password",
        "organization": "YourOrganization",
        "tenantName": "YourTenantName"
    },
    "script_config" : {
        "carolAppName": "myCarolAppName",
        "carolAppId": "carolAppIdFromDevTenant",
        "latestVersion": "0.0.32",
        "tenantsToUpdate": [
          "tenant1",
          "tenant2"
        ]
    }
}
```
`You need TENANT ADMIN permissions to run this script.`

#### REQUIREMENTS
`These are the packages the script needs before execution.`

In [ ]:
%%capture
!pip install --quiet pycarol
from pycarol import PwdAuth, Carol
import json, sys

#### CAROL LOGIN FUNCTIONS
`These are the functions made to login into Carol. They will be the same for all notebooks (ideally) and will use pyCarol.` 

[pyCarol reference](https://github.com/totvslabs/pyCarol)

In [ ]:
def carol_connect(username, password, organization, tenantName):
    print(f"Connecting to Carol tenant {tenantName}... ", end="\n")

    return Carol(domain=tenantName,
                auth=PwdAuth(username, password), organization=organization)

#### SCRIPT FUNCTIONS
`If the script requires more functions to execute, they will be here.`

In [ ]:
def check_installed_apps(carolObject):
    uri = f"v1/tenantApps?pageSize=-1"

    try:
        return carolObject.call_api(uri, "GET")
    except:
        return {"error" : str(sys.exc_info()[1])}

def publish_app(carolAppId, carolObject):
    uri = f"v1/tenantApps/{carolAppId}/publish"

    try:
        return carolObject.call_api(uri, "POST")
    except:
        return {"error" : str(sys.exc_info()[1])}

#### CONFIGURATION FILE
`Now you will need to upload the configuration file with the format given above.`

In [ ]:
try:
    from google.colab import files
    config_file = files.upload()
    config_json = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
    config_json_print = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
except:
    with open('./carol.json') as config_file:
        config_json = json.loads(config_file.read())
        config_json_print = json.loads(config_file.read())
    config_file.close()
finally:
    del config_json_print['authentication_config']['password']
    print(json.dumps(config_json_print, indent=2))

#### SCRIPT EXECUTION
`The main execution of the script will happen here.`

In [ ]:
for tenant in config_json['script_config']['tenantsToUpdate']:
    print(f'Starting process for {tenant}...')

    Carol = carol_connect(
    config_json['authentication_config']['username'], 
    config_json['authentication_config']['password'], 
    config_json['authentication_config']['organization'], 
    tenant)

    apps_installed = check_installed_apps(Carol)
    if apps_installed['count'] > 0:
      for app in apps_installed['hits']:
        if app['mdmName'] == config_json['script_config']['carolAppName'] and app['mdmInstallationTaskStatus'] == 'COMPLETED' and app['mdmAppVersion'] == config_json['script_config']['latestVersion']:
          try:
            publish_app(app['mdmId'], Carol)
            print(f'Tenant {tenant} published app {app["mdmName"]} version {app["mdmAppVersion"]}!')
          except:
            print(f'Something went wrong... e.g: Installation not completed, wrong version of CarolApp...')